Generate xyz files from smiles and fetch some experimental properties
---------------------------------------------------------------------
A code with different functionalities to convert a list of smiles stored in an excel sheet to xyz while storing multiple related details in the 2nd line of each of the xyz files
The scheme of the 2nd line works as follow:
* molecule name
* molecular formula 
* molecular weight (g/mol)
* experimental boiling point (degC)
* experimental melting point (degC)

###Used dependicies:
-------------------
Here, I used the awesome Leruli tool and rdkit <br>
for more about Leruli, please check https://www.leruli.com/ installable using: pip install leruli <br>
for more about rdkit, please check https://www.rdkit.org/   installable using: pip install rdkit
for more about plams, please check https://www.scm.com/doc/plams/index.html installable using: pip install plams

ofcourse, those functionalities are implemented in so many libraries, I just find those tools handy

What I like in Leruli is that it gives so many useful information in a efficient way and free of charge


###example:
--------
Aspirin.xyz
<br>14
<br>Aspirin C1COCCO1 88.05 101.605 -16.565
<br>C -0.731 1.16 0.225
<br>C 0.732 1.16 -0.225
<br>O 1.393 -0.0 0.236
<br>C 0.732 -1.16 -0.223
<br>C -0.733 -1.16 0.223
<br>O -1.393 0.001 -0.235
<br>H -0.774 1.202 1.324
<br>H -1.268 2.011 -0.196
<br>H 0.775 1.201 -1.324
<br>H 1.27 2.01 0.196
<br>H 1.268 -2.01 0.2
<br>H 0.777 -1.205 -1.322
<br>H -1.27 -2.009 -0.201
<br>H -0.778 -1.204 1.322

Please feel free to recommend other tools or corrections to this code!

In [1]:
import os
import leruli
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
from scm.plams import *

In [2]:
#create a dictionary of molecule name and their corresponding SMILES
smiles =    {'Methane'       : 'C',
            'Nitrobenzene'   : 'c1c(N(=O)=O)cccc1',
            'Benzene'        : 'c1ccccc1',
            'pyridine'       : 'n1ccccc1',
            'Aspirin'        : 'C1COCCO1',
            'Decalin'        : 'C1CCC2CCCCC2C1',
            'Toluene'        : 'C1=C(C)C=CC=C1'}

In [16]:
# I just prefer dataframe in showing the data in a nice format without much coding!
df = pd.DataFrame.from_dict(smiles, orient='index', columns=['smiles'])
df.index.name = 'molecule'

In [17]:
df

,smiles
molecule,
Methane,C
Nitrobenzene,c1c(N(=O)=O)cccc1
Benzene,c1ccccc1
pyridine,n1ccccc1
Aspirin,C1COCCO1
Decalin,C1CCC2CCCCC2C1
Toluene,C1=C(C)C=CC=C1


A function to convert SMILES to the corresponding chemical formula using leruli package

In [4]:
def smiles_to_formula(smiles):
    """
    Convert a list of SMILES strings to a list of their formulas.
    """
    formula_list = []
    for smile in smiles:
        formula_dict = leruli.graph_to_formula(smile)
        formula_list.append(formula_dict['formula'])
    return formula_list

Let's convert the SMILES to formulas using the function above implemented using leruli

In [5]:
df['formula'] = smiles_to_formula(df['smiles'])
df

,smiles,formula
molecule,,
Methane,C,CH4
Nitrobenzene,c1c(N(=O)=O)cccc1,C6H5NO2
Benzene,c1ccccc1,C6H6
pyridine,n1ccccc1,C5H5N
Aspirin,C1COCCO1,C4H8O2
Decalin,C1CCC2CCCCC2C1,C10H18
Toluene,C1=C(C)C=CC=C1,C7H8


A function to convert the SMILES to their respective experimental boiling point using leruli

In [6]:
def smiles_to_boiling_point(smiles):
    """
    Convert a list of SMILES strings to a list of their experimental boiling points.
    """
    smiles_without_bp = [] #list of SMILES strings without boiling point data available from Leruli API
    boiling_point_list = []
    for smile in smiles:
        if smile in smiles_without_bp:
            #set the bp to zero if the bp is not available in Leruli
            #I implemented it this way as didn't manage yet how to handle exceptions from the Leruli API
            boiling_point_dict = {'bp': 0} 
        else:
            boiling_point_dict = leruli.graph_to_boiling_point(smile)
        boiling_point_list.append(boiling_point_dict['bp'])
    return boiling_point_list

Let's add the corresponding boiling point to the dataframe

In [7]:
df['bp'] = smiles_to_boiling_point(df['smiles'])
df

,smiles,formula,bp
molecule,,,
Methane,C,CH4,-161.359
Nitrobenzene,c1c(N(=O)=O)cccc1,C6H5NO2,210.848
Benzene,c1ccccc1,C6H6,80.085
pyridine,n1ccccc1,C5H5N,115.282
Aspirin,C1COCCO1,C4H8O2,101.605
Decalin,C1CCC2CCCCC2C1,C10H18,179.544
Toluene,C1=C(C)C=CC=C1,C7H8,110.670


and the same for melting point

In [8]:
#define a function to convert SMILES to melting points
def smiles_to_melting_point(smiles):
    """
    Convert a list of SMILES strings to a list of their experimental melting points.
    """
    melting_point_list = []
    for smile in smiles:
        melting_point_dict = leruli.graph_to_melting_point(smile)
        melting_point_list.append(melting_point_dict['mp'])
    return melting_point_list

Lets get the melting points

In [9]:

df['mp'] = smiles_to_melting_point(df['smiles'])
df

,smiles,formula,bp,mp
molecule,,,,
Methane,C,CH4,-161.359,-182.362
Nitrobenzene,c1c(N(=O)=O)cccc1,C6H5NO2,210.848,5.755
Benzene,c1ccccc1,C6H6,80.085,4.748
pyridine,n1ccccc1,C5H5N,115.282,-41.558
Aspirin,C1COCCO1,C4H8O2,101.605,-16.565
Decalin,C1CCC2CCCCC2C1,C10H18,179.544,-33.921
Toluene,C1=C(C)C=CC=C1,C7H8,110.670,-93.283


In [10]:
#calculate the molecular weight of each molecule from smiles
def smiles_to_molecular_weight(smiles):
    """
    Convert a list of SMILES strings to a list of their molecular weights.
    """
    molecular_weight_list = []
    for smile in smiles:
        molecular_weight = ExactMolWt(Chem.MolFromSmiles(smile))
        molecular_weight = round(molecular_weight, 2)
        molecular_weight_list.append(molecular_weight)
    return molecular_weight_list

In [11]:
#lets get the molecular weight of each molecule
df['mw'] = smiles_to_molecular_weight(df['smiles'])
df

,smiles,formula,bp,mp,mw
molecule,,,,,
Methane,C,CH4,-161.359,-182.362,16.03
Nitrobenzene,c1c(N(=O)=O)cccc1,C6H5NO2,210.848,5.755,123.03
Benzene,c1ccccc1,C6H6,80.085,4.748,78.05
pyridine,n1ccccc1,C5H5N,115.282,-41.558,79.04
Aspirin,C1COCCO1,C4H8O2,101.605,-16.565,88.05
Decalin,C1CCC2CCCCC2C1,C10H18,179.544,-33.921,138.14
Toluene,C1=C(C)C=CC=C1,C7H8,110.670,-93.283,92.06


In [12]:
# %%
#make a new folder and name it xyz_structures
structure_folder = 'resultant_xyz_structures'
if not os.path.exists('./' + structure_folder):
    os.makedirs('./' + structure_folder)



Let's store the info we extracted in the 2nd line of each xyz structure as 
molecule nmae, SMILE, molecular weight, boiling point, melting point

<br> this can be useful if you prefer to quickly open a file and check related properties

In [13]:
xyz_path = './' + structure_folder + '/'
for index, row in df.iterrows():
    xyz_format = leruli.graph_to_geometry(graph=row['smiles'], format='XYZ')['geometry']
    with open(xyz_path + index +'.xyz', 'w') as f:
        f.write(xyz_format)
    with open(xyz_path + index +'.xyz', 'r') as f:
        lines = f.readlines()
        xyz_file = index + '.xyz'
        molecule_name = xyz_file.split('.')[0]
        lines[1] = molecule_name + ' ' + row['smiles'] + ' ' + str(row['mw']) + ' ' + str(row['bp']) + ' ' + str(row['mp']) + '\n'
        with open(xyz_path + index +'.xyz', 'w') as f:
            f.writelines(lines)
    

Creating xyz structures can be done also using plams package, and ofcourse many other libraries

In [14]:
#loop over the dataframe and create xyz files for each molecule
xyz_path = './' + structure_folder + '/'
for index, row in df.iterrows():
    plams_mol = from_smiles(row['smiles'])
    plams_mol.write(xyz_path + index +'_plams.xyz')

In [15]:
##re implemented above already
# #change the 2nd line in each xyz files within the xyz_structures to each dataframe row content separated by a space
# for xyz_file in os.listdir(xyz_path):
#     with open(xyz_path + xyz_file, 'r') as f:
#         lines = f.readlines()
#         lines[1] = str(row['mw']) + ' ' + str(row['bp']) + ' ' + str(row['mp']) + '\n'
#         #get the name of each file without the xyz extension
#         molecule_name = xyz_file.split('.')[0]
#         lines[1] = molecule_name + ' ' + row['smiles']+ ' ' + lines[1]
#         with open(xyz_path + xyz_file, 'w') as f:
#             f.writelines(lines)